In [1]:
import re
import os
import glob
import random 
import itertools

import numpy as np
import pandas as pd

from pathlib import Path
from scipy import signal
from scipy.io import loadmat

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split, Subset

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_encoder = preprocessing.LabelEncoder()

In [2]:
import wandb
# WandBの設定
WANDB_API_KEY = "2d996a98ef8dddefa91d675f85b5efd96fb911ae"  # あなたのWandB APIキーをここに入力してください

wandb.login(key = WANDB_API_KEY)

wandb: Currently logged in as: nekodaisuki169 (doctor_thesis_material). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/sota/.netrc


True

In [3]:


_FILENAME_RE = re.compile(r"s(\d+)_u(\d+)_d(\d+)\.mat$", re.IGNORECASE)

def _parse_lyon_filename(path: str):
    base = os.path.basename(path)
    m = _FILENAME_RE.match(base)
    if m is None:
        raise ValueError(f"Unexpected filename format: {base} (expected s*_u*_d*.mat)")
    speaker = int(m.group(1))
    utterance = int(m.group(2))
    digit = int(m.group(3))
    return speaker, utterance, digit

class LyonDecimation128Dataset(Dataset):
    """
    Lyon_decimation_128 (TI 46 digit) dataset for PyTorch.

    Returns per file:
      x_time:  (T, 77) float32
      y_time:  (T, 10) float32 with -1/+1 coding (digit col = +1, others = -1)
      y_seq:   (10,)  float32 one-hot (0/1)  ※デバッグ/他実装の互換用
      length:  int (T)
      label:   int (digit)
      meta:    dict (speaker, utterance, filename)
    """
    def __init__(self, dir_name: str, utterance_train_list, split: str = "train",
                 n_channel: int = 77, n_label: int = 10,
                 target_pm1: bool = True,
                 time_slice=None,  # (start, end) or None
                 debug_print_first: bool = False):
        super().__init__()
        self.dir_name = dir_name
        self.dir = Path(dir_name)
        self.utterance_train_list = set(utterance_train_list)
        self.split = split
        self.n_channel = n_channel
        self.n_label = n_label
        self.target_pm1 = target_pm1
        self.time_slice = time_slice
        self.debug_print_first = debug_print_first
        self.files = sorted(self.dir.glob("*.mat"))
        self.pattern = re.compile(r"s(\d+)_u(\d+)_d(\d+)\.mat$")

        self.speaker_ids = []
        self.utterance_ids = []
        self.digit_ids = []

        valid_files = []
        for p in self.files:
            m = self.pattern.search(p.name)
            if m is None:
                continue
            s = int(m.group(1))
            u = int(m.group(2))
            d = int(m.group(3))
            valid_files.append(p)
            self.speaker_ids.append(s)
            self.utterance_ids.append(u)
            self.digit_ids.append(d)

        paths = sorted(glob.glob(os.path.join(dir_name, "*.mat")))
        if not paths:
            raise RuntimeError(f"No .mat files found in: {dir_name}")

        items = []
        for p in paths:
            speaker, utterance, digit = _parse_lyon_filename(p)
            is_train = utterance in self.utterance_train_list
            if (split == "train" and is_train) or (split == "test" and (not is_train)):
                items.append((p, speaker, utterance, digit))

        if not items:
            raise RuntimeError(f"No files matched split='{split}'. Check utterance_train_list.")

        self.items = items

        # 長さ統計（デバッグ用）
        lens = []
        for (p, *_rest) in self.items[:20]:  # 最初の20個だけ軽く見る
            spec = loadmat(p)["spec"]  # expected (77, T)
            lens.append(int(spec.shape[1]))
        self._len_preview = lens

        if self.debug_print_first:
            print(f"[LyonDataset] split={split} n_files={len(self.items)} preview_T={self._len_preview}")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx: int):
        path, speaker, utterance, digit = self.items[idx]
        mat = loadmat(path)
        spec = mat["spec"]  # expected shape (77, T)

        if spec.shape[0] != self.n_channel:
            raise ValueError(f"spec.shape[0] != {self.n_channel}: got {spec.shape} in {path}")

        # (77, T) -> (T, 77)
        x = spec.T.astype(np.float32)

        # optional time slicing
        if self.time_slice is not None:
            s, e = self.time_slice
            x = x[s:e]

        T = x.shape[0]

        # y_time: (T, 10)
        if self.target_pm1:
            y = -np.ones((T, self.n_label), dtype=np.float32)
            y[:, digit] = 1.0
        else:
            # 0/1 encoding
            y = np.zeros((T, self.n_label), dtype=np.float32)
            y[:, digit] = 1.0

        # y_seq: (10,) (0/1)
        y_seq = np.zeros((self.n_label,), dtype=np.float32)
        y_seq[digit] = 1.0

        x_t = torch.from_numpy(x)              # (T, 77)
        y_t = torch.from_numpy(y)              # (T, 10)
        y_seq_t = torch.from_numpy(y_seq)      # (10,)
        length = T
        label = int(digit)

        meta = {"speaker": speaker, "utterance": utterance, "filename": os.path.basename(path)}

        return x_t, y_t, y_seq_t, length, label, meta



# class LyonDecimation128Dataset(Dataset):
#     def __init__(self, dir_name):
#         self.dir = Path(dir_name)
#         self.files = sorted(self.dir.glob("*.mat"))
#         if not self.files:
#             raise RuntimeError(f"No .mat files in {dir_name}")

#         self.pattern = re.compile(r"s(\d+)_u(\d+)_d(\d+)\.mat$")

#         self.speaker_ids = []
#         self.utterance_ids = []
#         self.digit_ids = []

#         valid_files = []
#         for p in self.files:
#             m = self.pattern.search(p.name)
#             if m is None:
#                 continue
#             s = int(m.group(1))
#             u = int(m.group(2))
#             d = int(m.group(3))
#             valid_files.append(p)
#             self.speaker_ids.append(s)
#             self.utterance_ids.append(u)
#             self.digit_ids.append(d)

#         self.files = valid_files
#         if not self.files:
#             raise RuntimeError("No valid Lyon files matched pattern s*_u*_d*.mat")

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, idx):
#         p = self.files[idx]
#         data = loadmat(p)
#         spec = data["spec"]                  # (77, T)
#         x = spec.T.astype(np.float32)        # (T,77)
#         T = x.shape[0]

#         x = torch.from_numpy(x).unsqueeze(0) # (1,T,77)

#         digit = self.digit_ids[idx]
#         y = -torch.ones(10, dtype=torch.float32)
#         y[digit] = 1.0

#         # ここで utterance なども返しておくとデバッグが楽
#         labels = {
#             "y": y,  # (10,)
#             "length": torch.tensor(T, dtype=torch.long),
#             "label_idx": torch.tensor(digit, dtype=torch.long),
#             "utterance": torch.tensor(self.utterance_ids[idx], dtype=torch.long),
#             "speaker": torch.tensor(self.speaker_ids[idx], dtype=torch.long),
#             "file": p.name,
#         }
#         return x, labels


def collate_pad_lyon(batch):
    # batch: list of (x_time(T,77), y_time(T,10), y_seq(10), length, label, meta)
    xs, ys, yseqs, lengths, labels, metas = zip(*batch)

    B = len(xs)
    Tmax = max(lengths)
    C_in = xs[0].shape[1]   # 77
    C_out = ys[0].shape[1]  # 10

    x_pad = torch.zeros((B, 1, Tmax, C_in), dtype=torch.float32)     # (B,1,T,77)
    y_pad = torch.zeros((B, Tmax, C_out), dtype=torch.float32)       # (B,T,10)
    mask  = torch.zeros((B, Tmax), dtype=torch.bool)

    # y の padding 値は -1 で埋める（pm1 前提なら重要）
    # ※ target_pm1=False のときは 0 埋めが自然。必要ならここを条件分岐してもOK。
    y_pad[:] = -1.0

    for i in range(B):
        T = lengths[i]
        x_pad[i, 0, :T, :] = xs[i]
        y_pad[i, :T, :] = ys[i]
        mask[i, :T] = True

    y_seq = torch.stack(yseqs, dim=0)            # (B,10)
    lengths_t = torch.tensor(lengths, dtype=torch.long)  # (B,)
    labels_t  = torch.tensor(labels, dtype=torch.long)   # (B,)

    return x_pad, y_pad, y_seq, lengths_t, labels_t, mask, metas


In [4]:
train_list = [1,2,3,4,5]
train_ds = LyonDecimation128Dataset("./Lyon_decimation_128", train_list, split="train",
                                    target_pm1=True, debug_print_first=True)
test_ds  = LyonDecimation128Dataset("./Lyon_decimation_128", train_list, split="test",
                                    target_pm1=True)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_pad_lyon)
test_loader  = DataLoader(test_ds, batch_size=8, shuffle=False, collate_fn=collate_pad_lyon)

# 1バッチだけ shape 確認
x_pad, y_pad, y_seq, lengths, labels, mask, metas = next(iter(train_loader))
print("x_pad   :", x_pad.shape)    # (B,1,Tmax,77)
print("y_pad   :", y_pad.shape)    # (B,Tmax,10)
print("y_seq   :", y_seq.shape)    # (B,10)
print("lengths :", lengths)        # (B,)
print("labels  :", labels)         # (B,)
print("mask    :", mask.shape, mask.sum().item())
print("meta[0] :", metas[0])


[LyonDataset] split=train n_files=250 preview_T=[49, 42, 49, 59, 57, 64, 71, 59, 50, 62, 49, 38, 44, 51, 57, 65, 73, 69, 53, 66]
x_pad   : torch.Size([8, 1, 95, 77])
y_pad   : torch.Size([8, 95, 10])
y_seq   : torch.Size([8, 10])
lengths : tensor([70, 95, 62, 83, 59, 79, 79, 48])
labels  : tensor([0, 9, 1, 7, 4, 6, 8, 2])
mask    : torch.Size([8, 95]) 575
meta[0] : {'speaker': 6, 'utterance': 2, 'filename': 's6_u2_d0.mat'}


In [5]:
class EchoStateNetwork(nn.Module):
    def __init__(self, model_params, dataset_params):
        super(EchoStateNetwork, self).__init__()
        
        self.reservoir_size = int(model_params["reservoir_size"])
        self.reservoir_weights_scale = float(model_params["reservoir_weights_scale"])
        
        self.input_size = int(model_params["input_size"])
        self.channel_size = int(model_params["channel_size"])
        self.input_weights_scale = float(model_params["input_weights_scale"])
        self.spectral_radius = float(model_params["spectral_radius"])
        self.density = float(model_params["reservoir_density"])
        self.leak_rate = float(model_params["leak_rate"])
        
        self.sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
        # self.sequence_length = int(dataset_params["seq_end"]) - int(dataset_params["seq_start"])

        # リザバー結合行列 (ランダムに初期化)
        self.register_parameter("reservoir_weights", torch.nn.Parameter(torch.empty((self.reservoir_size, self.reservoir_size)).uniform_(-self.reservoir_weights_scale, self.reservoir_weights_scale).to(device), requires_grad=False))
        
        # リザバー入力行列 (ランダムに初期化)
        self.register_parameter("input_weights", torch.nn.Parameter(torch.empty((self.reservoir_size, self.input_size * self.channel_size)).uniform_(-self.input_weights_scale, self.input_weights_scale).to(device), requires_grad=False))
        
        

        #リザバー結合のスパース処理
        self.reservoir_weights_mask = torch.empty((self.reservoir_size, self.reservoir_size)).uniform_(0, 1)
        self.reservoir_weights_mask = torch.where(self.reservoir_weights_mask < self.density, torch.tensor(1), torch.tensor(0)).to(device)
        self.reservoir_weights *= self.reservoir_weights_mask
        
        #スペクトル半径の処理
        _, singular_values, _ = torch.svd(self.reservoir_weights)
        rho_reservoir_weights = torch.max(singular_values).item()
        self.reservoir_weights *= self.spectral_radius / rho_reservoir_weights
        
       #最終時刻における，リザバー状態ベクトル
        self.last_reservoir_state_matrix = torch.zeros(self.channel_size, self.reservoir_size).to(device)
    

    def forward(self, x):
        # print(x.shape)
        batch_size = x.size(0)
        sequence_length = x.size(2)
        # x_seq = x.view(batch_size, self.channel_size, self.input_size, sequence_length).to(device)

        # if self.last_reservoir_state_matrix is None or self.last_reservoir_state_matrix.size(0) != batch_size:
        #     self.last_reservoir_state_matrix = torch.zeros(
        #         batch_size, self.channel_size, self.reservoir_size, device=device
        #     )
        
        # 各シーケンスのバッチに対してリザバー状態を初期化
        self.reservoir_state_matrix = torch.zeros(batch_size, self.channel_size,  sequence_length, self.reservoir_size).to(device)

        self.last_reservoir_state_matrix = torch.zeros(
                batch_size, self.channel_size, self.reservoir_size, device=device
            )
        
        for t in range(sequence_length):
            input_at_t = torch.matmul(x[:, :, t, :].view(batch_size, -1), self.input_weights.t())
            input_at_t = input_at_t.unsqueeze(1)

            if t == 0:
                state_update = torch.matmul(self.last_reservoir_state_matrix, self.reservoir_weights)
            else:
                # print("11111")
                state_update = torch.matmul(self.reservoir_state_matrix[:, :, t-1, :], self.reservoir_weights)
                # print(f"state_update.shape {state_update.shape}")
            self.reservoir_state_matrix[:, :, t, :] = self.leak_rate * torch.tanh(input_at_t + state_update) + \
                                                    (1 - self.leak_rate) * self.reservoir_state_matrix[:, :, t-1, :]

        

        self.last_reservoir_state_matrix = self.reservoir_state_matrix[:, :, -1, :]
        return self.reservoir_state_matrix

    def reset_hidden_state(self):
        self.last_reservoir_state_matrix = torch.zeros(self.channel_size, self.reservoir_size, device=device)
        # pass
        # print("内部状態がリセットされました")
    
#リードアウト層
class ReadOut(nn.Module):
    def __init__(self, model_params, dataset_params):
        super(ReadOut, self).__init__()
        # self.reservoir_state_matrix_size = int(model_params["reservoir_size"]) + int(model_params["input_size"]) + 1
        self.reservoir_state_matrix_size = int(model_params["reservoir_size"])
        self.output_size = int(model_params["ReadOut_output_size"])
        self.batch_training = model_params["Batch_Training"]
        self.channel_size = int(model_params["channel_size"])
        
        self.sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
        # self.sequence_length = int(dataset_params["seq_end"]) - int(dataset_params["seq_start"])
        
        self.readout_dense = nn.Linear(self.reservoir_state_matrix_size, self.output_size, bias=False)
        
        #線形回帰におけるバッチ学習を行うならば，リードアウト層を最急降下法による学習対象にしない
        if self.batch_training == True:
            self.readout_dense.weight.requires_grad = False
        else:
            None
            
        nn.init.xavier_uniform_(self.readout_dense.weight)
        
    def forward(self, x):
        # x shape: [batch_size, channel_size, input_size, sequence_length]
        # batch_size, channel_size, input_size, sequence_length = x.size()
        
        # Reshape x to apply the dense layer to each time step and channel independently
        # New shape: [batch_size * channel_size * sequence_length, input_size]
        # x_reshaped = x.permute(0, 1, 3, 2).contiguous().view(-1, input_size)
        
        # Apply the dense layer
        # Output shape: [batch_size * channel_size * sequence_length, output_size]
        # print(f"readout x shape{x.shape}")
        output = self.readout_dense(x)
        
        # Reshape the output back to the original form
        # Output shape: [batch_size, channel_size, output_size, sequence_length]
        # output = output.view(batch_size, channel_size, sequence_length, -1).permute(0, 1, 3, 2).contiguous()
        return output
    
    # リッジ回帰によるリードアウトの導出
    @staticmethod
    def ridge_regression(X, Y, alpha):
        # データ行列 X の形状を取得
        n, p = X.shape

        # 正則化項の行列を作成
        ridge_matrix = (alpha * torch.eye(n)).float().to(device)
        X = X.float()
        Y = Y.float()

        # リッジ回帰の係数を計算
        coefficients = torch.linalg.solve(torch.matmul(X, X.T) + ridge_matrix, torch.matmul(X, Y.T)).T

        return coefficients

    @staticmethod
    def ridge_regression_update(outputs, targets, model, alpha=0):
        with torch.no_grad():
            # リッジ回帰を用いて重みを求める
            # モデルのパラメータを更新
            new_weights = ReadOut.ridge_regression(outputs.squeeze(), targets.squeeze(), alpha)

            # モデルのパラメータを更新
            model.ReadOut.readout_dense.weight.copy_(new_weights)
        
        return None
    #それぞれのモデルパラメータ候補を辞書に格納する
    @staticmethod
    def model_params_candinate(model_params):
        model_params_combinations = list(itertools.product(*model_params.values()))
        param_dicts = [dict(zip(model_params.keys(), combination)) for combination in model_params_combinations]
        
        return param_dicts

    #モデル構造を辞書型に格納
    @staticmethod
    def model_sturcture_dict(model):
        layers_dict = {}
        for name, module in model.named_modules():
            layers_dict[name] = {
                'type': type(module).__name__,
                'parameters': {p: getattr(module, p) for p in module.__dict__ if not p.startswith('_')}
            }
        
        #モデル名と初期の引数は削除
        del(layers_dict[''])
        
        return layers_dict

class ESN(nn.Module):
    def __init__(self, model_params, training_params, dataset_params):
        super(ESN, self).__init__()
        self.ESN = EchoStateNetwork(model_params, dataset_params)
        self.ReadOut = ReadOut(model_params, dataset_params)
    
    def forward(self, x):
        self.Reservoir_State_Matrix = self.ESN(x)
        # print(f"resever state matrix {self.Reservoir_State_Matrix.shape}")
        self.ReadOut_Reservoir = self.ReadOut(self.Reservoir_State_Matrix)
        # print("重み:", self.ReadOut.readout_dense.weight)
        # print(self.ReadOut_Reservoir)

        #channle_size次元はいらないので，減らす
        self.ReadOut_Reservoir = self.ReadOut_Reservoir.squeeze(1)
        # print(f"output matrix {self.ReadOut_Reservoir.shape}")

        return self.ReadOut_Reservoir

    def reset_hidden_state(self):
        self.ESN.reset_hidden_state()

In [6]:


# K分割交差検証とデータローダーの生成
from torch.utils.data import DataLoader, Subset

def create_cross_validation_dataloaders(dataset, dataset_params, traing_params, train_list):
    """
    Lyon原コード準拠：
      train_list に含まれる utterance(u) → train
      それ以外 → val
    戻り値は (train_loader, val_loader) を1組だけ入れたリスト
    """
    batch_size = int(dataset_params["batch_size"])

    train_idx = [i for i, u in enumerate(dataset.utterance_ids) if u in train_list]
    val_idx   = [i for i, u in enumerate(dataset.utterance_ids) if u not in train_list]

    if len(train_idx) == 0 or len(val_idx) == 0:
        raise RuntimeError(f"Split error: train={len(train_idx)}, val={len(val_idx)}. train_list={train_list}")

    train_subset = Subset(dataset, train_idx)
    val_subset   = Subset(dataset, val_idx)

    # あなたの方針どおり “フルバッチ” にしたい場合は len(train_subset) を使う
    train_loader = DataLoader(
        train_subset,
        batch_size=len(train_subset),      # ←元コード踏襲（フルバッチ）
        shuffle=True,
        collate_fn=collate_pad_lyon
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=len(val_subset),        # ←元コード踏襲（フルバッチ）
        shuffle=False,
        collate_fn=collate_pad_lyon
    )

    # 互換性のため list で返す（fold=0 だけ回る）
    return [(train_loader, val_loader)]

def prepare_datasets(dataset_params, traing_params, data_dir, train_list):
    dataset = LyonDecimation128Dataset(data_dir, train_list)

    cross_val_loaders = create_cross_validation_dataloaders(
        dataset, dataset_params, traing_params, train_list
    )

    # Lyon原コードは明確な test を別に持たないので、ここでは None にしておくのが安全
    test_loader = None
    return cross_val_loaders, test_loader

def debug_split(dataset, train_list):
    from collections import Counter
    train_u = [u for u in dataset.utterance_ids if u in train_list]
    val_u   = [u for u in dataset.utterance_ids if u not in train_list]
    print("[DEBUG] total:", len(dataset))
    print("[DEBUG] train:", len(train_u), "val:", len(val_u))
    print("[DEBUG] train utterance counts:", Counter(train_u))
    print("[DEBUG] val utterance counts  :", Counter(val_u))

In [7]:
def unpack_batch_for_lyon(batch):
    """
    train_loader が返す batch の形式が複数あり得るので吸収する

    return:
      inputs: Tensor (B,1,T,F)
      labels: Tensor (B,C)  ※one-hot or ±1
      lengths: Tensor (B,) or None
      mask: Tensor (B,T) bool or None
      label_idx: Tensor (B,) long or None
      extra: 何かあれば（ファイル名など）
    """
    if isinstance(batch, (list, tuple)):
        if len(batch) == 2:
            inputs, labels = batch
            return inputs, labels, None, None, None, None

        # よくある Lyon 可変長形式
        if len(batch) >= 5:
            inputs = batch[0]
            labels = batch[1]
            lengths = batch[2]
            mask = batch[3]
            label_idx = batch[4]
            extra = batch[5:] if len(batch) > 5 else None
            return inputs, labels, lengths, mask, label_idx, extra

    raise ValueError(f"Unexpected batch format: type={type(batch)}, batch={batch}")


def _debug_check_view_mapping(outputs, outputs_flatten, prefix=""):
    """
    outputs: (B,T,C)
    outputs_flatten: (C, B*T) を想定
    期待対応: outputs_flatten[:, b*T + t] == outputs[b,t,:]
    """
    if outputs.dim() != 3 or outputs_flatten.dim() != 2:
        print(prefix, "skip mapping check (dim mismatch)")
        return

    B, T, C = outputs.shape
    if outputs_flatten.shape[0] != C:
        print(prefix, f"skip mapping check: outputs_flatten[0]={outputs_flatten.shape[0]} != C={C}")
        return

    # いくつかサンプル点を比較
    check_points = [(0,0), (0,1), (0, min(5, T-1))]
    if B >= 2:
        check_points += [(1,0), (1, min(5, T-1))]

    for (b,t) in check_points:
        j = b*T + t
        if j >= outputs_flatten.shape[1]:
            continue
        diff = (outputs[b, t, :] - outputs_flatten[:, j]).abs().max().item()
        print(prefix, f"view-map check (b={b},t={t},j={j}) max|diff|={diff:.6g}")



In [8]:
def train_model(model, criterion, optimizer, train_loader, dataset_params, model_params, traing_params):
    model.train()
    num_epochs = int(traing_params["num_epochs"])
    reservoir_size = int(model_params["reservoir_size"])
    batch_training = model_params["Batch_Training"]
    Regularization_L2 = model_params["Regularization_L2"]

    for epoch in range(num_epochs):
        running_loss = 0.0

        for it, batch in enumerate(train_loader):
            inputs, labels, lengths, mask, label_idx, extra = unpack_batch_for_lyon(batch)

            inputs = inputs.float().to(device)

            # labels が dict のケースにも対応（あなたがどっちで返しててもOKにする）
            if isinstance(labels, dict):
                y = labels["y"].to(device)               # (B,C)
                if mask is None and "mask" in labels:    # loader側で dict に入れてる場合
                    mask = labels["mask"]
                if lengths is None and "lengths" in labels:
                    lengths = labels["lengths"]
                if label_idx is None and "label_idx" in labels:
                    label_idx = labels["label_idx"]
            else:
                y = labels.to(device)                    # (B,C)

            if mask is not None:
                mask = mask.to(device)                   # (B,T) bool
            if lengths is not None:
                lengths = lengths.to(device)

            optimizer.zero_grad()

            outputs_ESN = model.ESN(inputs)
            outputs = model.ReadOut(outputs_ESN)
            outputs = outputs.squeeze()                  # (B,T,C) を期待

            if outputs.dim() != 3:
                raise ValueError(f"[train] outputs must be (B,T,C), got {outputs.shape}")

            B, T, C = outputs.shape

            # ===== デバッグ表示（最初の1バッチだけ）=====
            if epoch == 0 and it == 0:
                print("===== [DEBUG train first batch] =====")
                print("batch type/len:", type(batch), len(batch) if isinstance(batch,(list,tuple)) else "N/A")
                print("inputs      :", tuple(inputs.shape), "contig=", inputs.is_contiguous())
                print("y           :", tuple(y.shape))
                print("outputs_ESN :", tuple(outputs_ESN.shape), "contig=", outputs_ESN.is_contiguous())
                print("outputs     :", tuple(outputs.shape), "contig=", outputs.is_contiguous())
                if mask is not None:
                    print("mask        :", tuple(mask.shape), "valid_frames=", int(mask.sum().item()))
                if lengths is not None:
                    print("lengths[:8] :", lengths[:min(8,B)].tolist())
                print("=====================================")

            # ===== 可変長対応：padding を無効化（viewの前に掛ける）=====
            if mask is not None:
                mask_f = mask.float()
                outputs = outputs * mask_f.unsqueeze(-1)

                # outputs_ESN にもマスク（形状に応じて）
                if outputs_ESN.dim() == 4:         # (B,1,T,H)
                    outputs_ESN = outputs_ESN * mask_f.unsqueeze(1).unsqueeze(-1)
                elif outputs_ESN.dim() == 3:       # (B,T,H)
                    outputs_ESN = outputs_ESN * mask_f.unsqueeze(-1)

            # ---- flattenの整合性（ここはあなたの元コードを維持）----
            outputs_ESN_flatten = outputs_ESN.view(reservoir_size, -1)
            outputs_flatten     = outputs.view(C, -1)

            labels_rep = y.unsqueeze(1).repeat(1, T, 1)   # (B,T,C)

            if mask is not None:
                labels_rep = labels_rep * mask_f.unsqueeze(-1)

            labels_flatten = labels_rep.view(C, -1)

            if epoch == 0 and it == 0:
                print("[DEBUG] outputs_flatten:", tuple(outputs_flatten.shape))
                print("[DEBUG] labels_flatten :", tuple(labels_flatten.shape))
                if lengths is not None:
                    Ti = int(lengths[0].item())
                    tail_energy = outputs[0, Ti:, :].abs().sum().item()
                    print(f"[DEBUG] tail_energy(sample0 after Ti={Ti}) = {tail_energy:.6g} (should be ~0 if mask ok)")

            loss = criterion(outputs_flatten, labels_flatten)

            if batch_training == True:
                model.ReadOut.ridge_regression_update(outputs_ESN_flatten, labels_flatten, model, Regularization_L2)
            else:
                loss.backward()
                optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
        wandb.log({"loss": epoch_loss})

    print("Training complete")

def validate_model(model, val_loader, dataset_params, model_params):
    model.eval()
    val_running_loss = 0.0

    with torch.no_grad():
        for it, batch in enumerate(val_loader):
            inputs, labels, lengths, mask, label_idx, extra = unpack_batch_for_lyon(batch)

            inputs = inputs.float().to(device)

            if isinstance(labels, dict):
                y = labels["y"].to(device)
                if mask is None and "mask" in labels:
                    mask = labels["mask"]
                if lengths is None and "lengths" in labels:
                    lengths = labels["lengths"]
            else:
                y = labels.to(device)

            if mask is not None:
                mask = mask.to(device)
            if lengths is not None:
                lengths = lengths.to(device)

            outputs_ESN = model.ESN(inputs)
            outputs = model.ReadOut(outputs_ESN).squeeze(1)  # ←あなたの元コード維持

            if outputs.dim() != 3:
                raise ValueError(f"[val] outputs must be (B,T,C), got {outputs.shape}")

            B, T, C = outputs.shape

            if it == 0:
                print("===== [DEBUG val first batch] =====")
                print("inputs      :", tuple(inputs.shape))
                print("y           :", tuple(y.shape))
                print("outputs_ESN :", tuple(outputs_ESN.shape))
                print("outputs     :", tuple(outputs.shape))
                if mask is not None:
                    print("mask        :", tuple(mask.shape), "valid_frames=", int(mask.sum().item()))
                if lengths is not None:
                    print("lengths[:8] :", lengths[:min(8,B)].tolist())
                print("===================================")

            # padding無効化（viewの前）
            if mask is not None:
                mask_f = mask.float()
                outputs = outputs * mask_f.unsqueeze(-1)

            outputs_flatten = outputs.view(C, -1)
            labels_rep = y.unsqueeze(1).repeat(1, T, 1)
            if mask is not None:
                labels_rep = labels_rep * mask_f.unsqueeze(-1)
            labels_flatten = labels_rep.view(C, -1)

            loss = criterion(outputs_flatten, labels_flatten)  # ←元コード同様 global criterion を使う
            val_running_loss += loss.item() * B

    val_loss = val_running_loss / len(val_loader.dataset)

    # Lyon(可変長)のときは sequence_length 固定 split が破綻するので、
    # まずは loss のみで確認するのが安全です（精度は後で“発話単位投票”に差し替え推奨）
    print(f"Validation Loss: {val_loss:.4f}")
    wandb.log({"val_loss": val_loss})


    
def test_model(model, val_loader):
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    val_running_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.squeeze().to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
#             _, label_preds = torch.max(labels, 1)
            label_preds = labels

            val_running_loss += loss.item() * inputs.size(0)
            print(loss.item())
            val_running_corrects += torch.sum(preds == label_preds)

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = val_running_corrects.double() / len(val_loader.dataset)
    
    print(f'Test Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
    wandb.log({"test_accuracy": val_accuracy, "test_loss": val_loss})

def compute_accuracy(model_output, target, n_taus):
    # モデルの出力と教師ラベルを各データに分割
    split_model_output = torch.split(model_output.squeeze(), n_taus, dim=-1)
    split_target = torch.split(target.squeeze(), n_taus, dim=-1)
    correct = 0
    total = 0
    
    for pred, true_label in zip(split_model_output, split_target):
        # 最も確率が高いラベルを予測ラベルとして取得
        # print(pred.shape)
        # print(pred)
        # print(true_label.shape)
        # print(true_label)
        count_ones = (true_label == 1).sum().item()
        # print(count_ones)
        histgram_predict = torch.bincount(torch.max(pred, 0)[1])
        _, predicted = torch.max(histgram_predict, 0)
    
        histgram_true_label_idx = torch.bincount(torch.max(true_label, 0)[1])
        _, true_label_idx = torch.max(histgram_true_label_idx, 0)

        # 正解数をカウント
        correct += (predicted == true_label_idx).sum().item()
        total += 1

    # 精度を算出
    accuracy = correct / total
    return accuracy

def model_params_candinate(model_params):
    model_params_combinations = list(itertools.product(*model_params.values()))
    param_dicts = [dict(zip(model_params.keys(), combination)) for combination in model_params_combinations]
    return param_dicts

# モデル構造を辞書型に格納
def model_sturcture_dict(model):
    layers_dict = {}
    for name, module in model.named_modules():
        layers_dict[name] = {
            'type': type(module).__name__,
            'parameters': {p: getattr(module, p) for p in module.__dict__ if not p.startswith('_')}
        }
    # モデル名と初期の引数は削除
    del(layers_dict[''])
    return layers_dict
    

In [9]:
train_list = [1,2,3,4,5]

#各種のパラメータ設定
dataset_params = {"seq_start" : 400, "seq_end" : 1200, "sequence_length": 800, "slicing_size" : 1, "augmentation_factor": 0, "batch_size" : 32, "Onehot_Encoding" : None, "augmentation_mu" : 0, "augmentation_sigma" : 0, "augmentation_shift" : 1}
model_params = {"reservoir_size" : [100],"input_size" : [77], "channel_size" : [1],  "reservoir_weights_scale" : [1], "input_weights_scale" : [1], "spectral_radius" : [0.9],"reservoir_density" : [0.05], "leak_rate" : [1], "Batch_Training" : [True], "ReadOut_output_size" : [10], "Regularization_L2" : [0.01]}
training_params = {"num_epochs" : 1, "learning_rate" : 0.01, "weight_decay" : 1e-2, "testdata_ratio" : 0, "n_splits" : 5}

#それぞれのモデルパラメータ候補を辞書に格納する
model_params = model_params_candinate(model_params)

#学習データセットの設定
data_dir="./Lyon_decimation_128/"
cross_val_loaders, test_loader = prepare_datasets(dataset_params, training_params, data_dir, train_list)

# wandb.init(project="uskin_test_ESN", config=config_dictionary)

#モデルパラメータの候補ごとに，総当たりしてパラメータを探索する

for each_model_params in model_params:
    
#     model = LSTMModel(each_model_params).to(device)
    model = ESN(each_model_params, training_params, dataset_params).to(device)
    model_sturcture = model_sturcture_dict(model)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = float(training_params["learning_rate"]), weight_decay = float(training_params["weight_decay"]))
    
    config_dictionary = {
    "dataset": data_dir,
    "dataset_params" : dataset_params,
    "architecture": model.__class__.__name__,
    "model_params" : each_model_params,
    "model_sturcture" : model_sturcture,
    "traing_params" : training_params,
    "criterion" : str(criterion),
    "optimizer" : str(optimizer),
    }

    wandb.init(project="Lyon_test_ESN", config=config_dictionary)

    # 4. k-fold交差検証のループ
    for fold, (train_loader, val_loader) in enumerate(cross_val_loaders):
        print(f'FOLD {fold}')
        print('--------------------------------')

        train_model(model, criterion, optimizer, train_loader, dataset_params, each_model_params, training_params)
        validate_model(model, val_loader,dataset_params, each_model_params)
        model.__init__(each_model_params, training_params, dataset_params)

    print('CrossVaridation Finished')
    print('--------------------------------')
    #テストデータによる評価
    # テストデータローダーの準備

#     test_model(model, test_loader)
    
    wandb.finish()

FOLD 0
--------------------------------


IndexError: list index out of range

In [ ]:
def print_trainable_parameters(model: nn.Module):
    print("\n===== Model Parameters =====")
    for name, param in model.named_parameters():
        print(
            f"{name:40s} | shape={tuple(param.shape)} | requires_grad={param.requires_grad}"
        )

def print_esn_layer_details(model: ESN):
    print("\n===== ESN Layer Details =====")

    # --- Echo State Network ---
    esn = model.ESN
    print("\n--- EchoStateNetwork ---")
    print(f"reservoir_size        : {esn.reservoir_size}")
    print(f"input_size            : {esn.input_size}")
    print(f"channel_size          : {esn.channel_size}")
    print(f"sequence_length       : {esn.sequence_length}")
    print(f"leak_rate             : {esn.leak_rate}")
    print(f"spectral_radius       : {esn.spectral_radius}")
    print(f"reservoir_density     : {esn.density}")
    print(f"input_weights_scale   : {esn.input_weights_scale}")
    print(f"reservoir_weights_scale: {esn.reservoir_weights_scale}")

    print("\n[Reservoir Weights]")
    print(f"shape         : {tuple(esn.reservoir_weights.shape)}")
    print(f"requires_grad : {esn.reservoir_weights.requires_grad}")

    print("\n[Input Weights]")
    print(f"shape         : {tuple(esn.input_weights.shape)}")
    print(f"requires_grad : {esn.input_weights.requires_grad}")

    print("\n[Last Reservoir State]")
    print(f"shape         : {tuple(esn.last_reservoir_state_matrix.shape)}")

    # --- ReadOut ---
    readout = model.ReadOut
    print("\n--- ReadOut ---")
    print(f"reservoir_state_matrix_size : {readout.reservoir_state_matrix_size}")
    print(f"output_size                 : {readout.output_size}")
    print(f"channel_size                : {readout.channel_size}")
    print(f"sequence_length             : {readout.sequence_length}")
    print(f"batch_training              : {readout.batch_training}")

    print("\n[ReadOut Dense Layer]")
    print(f"weight shape    : {tuple(readout.readout_dense.weight.shape)}")
    print(f"requires_grad   : {readout.readout_dense.weight.requires_grad}")
    print(f"bias            : {readout.readout_dense.bias}")

def print_model_structure(model: nn.Module):
    print("===== Model Structure (Hierarchy) =====")
    for name, module in model.named_modules():
        if name == "":
            continue
        print(f"[{name}] -> {module.__class__.__name__}")


In [ ]:
model = ESN(each_model_params, training_params, dataset_params).to(device)
print_model_structure(model)
print_esn_layer_details(model)